In [ ]:
import plotly
import quaternion
import numpy as np
import scipy.optimize as opt
import plotly.graph_objects as go

from scipy.integrate import solve_ivp
from IPython.display import display, HTML

plotly.offline.init_notebook_mode()
display(HTML('<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'))

In [ ]:
r_moon  = 1737.1e3
mu_moon = 4.9048695e12

In [ ]:
def gravity(r, mu):
    '''
    Calculates the gravitational acceleration at a point
    :param r: position vector
    :param mu: gravitational parameter of the central body
    :return: gravitational acceleration vector
    '''
    return -mu*r/(np.linalg.norm(r)**3)

In [ ]:
def lla_to_cart(lla_pos, R):
    lat = np.radians(lla_pos[0])
    lon = np.radians(lla_pos[1])
    alt = lla_pos[2]

    x = R * np.cos(lat) * np.cos(lon)
    y = R * np.cos(lat) * np.sin(lon)
    z = R * np.sin(lat) + alt
    return np.array([x, y, z])

def cart_to_lla(cart_pos, R):
    x = cart_pos[0]
    y = cart_pos[1]
    z = cart_pos[2]
    
    lat = np.arcsin(z/R)
    lon = np.arctan2(y, x)
    alt = np.linalg.norm(cart_pos) - R
    return np.array([np.degrees(lat), np.degrees(lon), alt])

def get_inertial_to_enu(lla_pos):
    lat = np.radians(lla_pos[0])
    lon = np.radians(lla_pos[1])
    R_mcmf_to_enu = np.array([[-np.sin(lon),              np.cos(lon),                0],
                              [-np.sin(lat)*np.cos(lon),  -np.sin(lat)*np.sin(lon),   np.cos(lat)],
                              [np.cos(lat)*np.cos(lon),   np.cos(lat)*np.sin(lon),    np.sin(lat)]])
    return quaternion.from_rotation_matrix(R_mcmf_to_enu)

def convert_inertial_state_to_enu(pos, lla_ref):
    pos_ref = lla_to_cart(lla_ref, r_moon)
    q_inertial_to_enu = get_inertial_to_enu(lla_ref)
    pos_enu = quaternion.rotate_vectors(q_inertial_to_enu, pos-pos_ref)
    return pos_enu

def get_inertial_to_lvlh(state):
    '''
    Calculates the rotation matrix from the LVLH frame to the inertial frame
    :param state: current state vector [x, y, z, vx, vy, vz]
    :return: rotation matrix from the LVLH frame to the inertial frame
    '''
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)

    r_normalized    = r/np.linalg.norm(r)
    h_normalized    = h/np.linalg.norm(h)
 
    # Rotation matrix from Hill to Inertial
    R_lvlh_to_inertial = np.array([np.cross(-h_normalized, -r_normalized), -h_normalized, -r_normalized]).T
    q_lvlh_to_inertial = quaternion.from_rotation_matrix(R_lvlh_to_inertial)
    q_inertial_to_lvlh = q_lvlh_to_inertial.conjugate()

    return q_inertial_to_lvlh

In [ ]:
def E_func(E, M, e):
        return E - e*np.sin(E) - M
def E_prime(E, M, e):
    return 1 - e*np.cos(E)

def eccentric_anomaly(M, e):
    E = opt.newton(E_func, M+e, fprime=E_prime, args=(M,e),tol=1e-10,)
    return E

def mean_to_true_anomaly(M, e):
    E = eccentric_anomaly(M, e)
    f = 2*np.arctan2(np.tan(E/2)*np.sqrt(1+e), np.sqrt(1-e))
    if f > np.pi:
        f -= 2*np.pi
    if f < -np.pi:
        f += 2*np.pi
    return f
def disp_state_vec(state):
    print(f"R:\t [{state[0]}, {state[1]}, {state[2]}] m")
    print(f"V:\t [{state[3]}, {state[4]}, {state[5]}] m/s\n")

def disp_kep_elem(kep):
    print(f"Semimajor Axis (a):\t\t {kep[0]} m")
    print(f"Eccentricity (e):\t\t {kep[1]}")
    print(f"Inclination (i):\t\t {np.degrees(kep[2])}º")
    print(f"RAAN (Ω):\t\t\t {np.degrees(kep[3])}º")
    print(f"Argument of Periapse (𝜔):\t {np.degrees(kep[4])}º")
    print(f"True Anomaly (f):\t\t {np.degrees(kep[5])}º\n")

def state_vec_to_keplerian(state, mu):
    a = semimajor_axis(state, mu)
    e = np.linalg.norm(eccentricity(state, mu))
    i = inclination(state)
    raan = RAAN(state)
    omega = argument_of_periapse(state, mu)
    f = true_anomaly(state, mu)
    return np.array([a, e, i, raan, omega, f])

def keplerian_to_state_vec(kep, mu):
    pos = position(kep, mu)
    vel = velocity(kep, mu)

    state_vec = np.array([pos[0], pos[1], pos[2], vel[0], vel[1], vel[2]])
    disp_state_vec(state_vec)
    input = kepDeg2Rad(kep)
    check = state_vec_to_keplerian(state_vec, mu)
    if np.allclose(input, check, rtol=1e-04, atol=1e-05):
        return state_vec
    else:
        print("WARNING: Keplerian elements do not match")
        disp_kep_elem(input)
        disp_kep_elem(check)
        return state_vec

def semimajor_axis(state, mu):
    r = np.linalg.norm(state[0:3])
    v = np.linalg.norm(state[3:6])
    if r == 0:
        return 0
    a = 1/(2/r - v**2/mu)
    return a

def eccentricity(state, mu):
    r = state[0:3]
    v = state[3:6]
    r_mag = np.sqrt(np.dot(r, r))
    v_mag = np.sqrt(np.dot(v, v))
    if r_mag == 0:
        return 0
    e = (v_mag**2/mu - 1/r_mag)*r - 1/mu*np.dot(r, v)*v
    return e

def inclination(state):
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)
    h_mag = np.sqrt(np.dot(h, h))
    if h_mag == 0:
        return 0
    incl = np.arccos(np.dot(h/h_mag, np.array([0, 0, 1])))
    return incl

def node_vector(state):
    r = state[0:3]
    v = state[3:6]
    h = np.cross(r, v)
    n = np.cross(np.array([0, 0, 1]), h)
    return n

def RAAN(state):
    node = node_vector(state)
    node_mag = np.sqrt(np.dot(node, node))
    if node_mag == 0:
        return 0
    omega = np.arccos(np.dot(node, np.array([1, 0, 0])) / node_mag)
    if np.dot(node, np.array([0, 1, 0])) < 0:
        omega = 2*np.pi - omega
    return omega

def argument_of_periapse(state, mu):
    node = node_vector(state)
    node_mag = np.sqrt(np.dot(node, node))
    e = eccentricity(state, mu)
    e_mag = np.sqrt(np.dot(e, e))
    if node_mag == 0 or e_mag == 0:
        return 0
    omega = np.arccos(np.dot(node, e)/(node_mag*e_mag))
    if (np.dot(e, np.array([0, 0, 1])) < 0):
        omega = 2*np.pi - omega
    return omega

def true_anomaly(state, mu):
    r = state[0:3]
    e = eccentricity(state, mu)
    r_mag = np.sqrt(np.dot(r, r))
    e_mag = np.sqrt(np.dot(e, e))
    if r_mag == 0 or e_mag == 0:
        return 0
    if np.isclose(np.dot(e, r)/(e_mag*r_mag), 1):
        return 0
    f = np.arccos(np.dot(e, r)/(e_mag*r_mag))
    while f >= 2*np.pi:
        f -= 2*np.pi
    while f <= -2*np.pi:
        f += 2*np.pi
    return f

def angular_momentum_from_OE(a, e, mu):
    if e >=0 and e < 1:
        return np.sqrt(mu*a*(1-e**2))
    elif e < 0:
        return np.sqrt(mu*a*(e**2-1))


def position(kep, mu):
    a = kep[0]
    e = kep[1]
    i = np.radians(kep[2])
    raan = np.radians(kep[3])
    omega = np.radians(kep[4])
    f = np.radians(kep[5])

    r = a*(1-e**2)/(1+e*np.cos(f))
    theta = omega + f
    pos = r * np.array([np.cos(theta)*np.cos(raan) - np.cos(i)*np.sin(raan)*np.sin(theta),
                        np.cos(theta)*np.sin(raan) + np.cos(i)*np.cos(raan)*np.sin(theta),
                        np.sin(i)*np.sin(theta)])
    return pos

def velocity(kep, mu):
    a = kep[0]
    e = kep[1]
    inc = np.radians(kep[2])
    raan = np.radians(kep[3])
    omega = np.radians(kep[4])
    f = np.radians(kep[5])

    theta = omega + f

    h = angular_momentum_from_OE(a, e, mu)
    theta = omega + f
    vel_vec = mu/h*np.array([-(np.cos(raan)*(np.sin(theta) + e*np.sin(omega)) + np.sin(raan)*(np.cos(theta)+ e*np.cos(omega))*np.cos(inc)),
                             -(np.sin(raan)*(np.sin(theta) + e*np.sin(omega)) - np.cos(raan)*(np.cos(theta) + e*np.cos(omega))*np.cos(inc)),
                             (np.cos(theta) + e*np.cos(omega))*np.sin(inc)])              
    return vel_vec

def kepRad2Deg(kep):
    return np.array([kep[0], kep[1], np.degrees(kep[2]), np.degrees(kep[3]), np.degrees(kep[4]), np.degrees(kep[5])])

def kepDeg2Rad(kep):
    return np.array([kep[0], kep[1], np.radians(kep[2]), np.radians(kep[3]), np.radians(kep[4]), np.radians(kep[5])])   

In [ ]:
def get_torque_cmd(q_inertial_to_body, omega, q_inertial_to_target):
    k_p = 7600
    k_d = 9550
    q_i2b   = quaternion.from_float_array(q_inertial_to_body)
    q_i2t   = quaternion.from_float_array(q_inertial_to_target)
    omega   = omega

    dq      = get_error_quat(q_i2b, q_i2t)
    dq_vec  = np.array([dq.x, dq.y, dq.z])

    torque_cmd = -k_p * dq_vec - k_d * omega
    return torque_cmd

def get_error_quat(q1, q2):
    if np.dot(quaternion.as_float_array(q1), quaternion.as_float_array(q2)) < 0:
        q1 = -q1
    return q1 * q2.inverse()

def get_error_quats(q1s, q2s):
    q_errs = np.zeros((len(q1s),4))
    for i in range(len(q1s)):
        q1          = quaternion.from_float_array(q1s[i])
        q2          = quaternion.from_float_array(q2s[i])
        dq          = get_error_quat(q1, q2)
        q_err       = quaternion.as_float_array(dq)
        q_errs[i]   = q_err
    return q_errs

def get_guidance_mode(time, state, current_mode, mode_switch_states, t_mode2):
    radial_rate = np.dot(state[0:3]/np.linalg.norm(state[0:3]), state[3:6])
    if current_mode == -1:
        return 0
    elif current_mode == 0 and time > mode_switch_states[0]:
        return 1
    elif current_mode == 1 and radial_rate > mode_switch_states[1]:
        return 2
    elif current_mode == 2 and time-t_mode2 > mode_switch_states[2]:
        return 3
    return current_mode
    
def get_sc_commands(t, state, state_init, guid_mode, guid_params, t_mode2):
    q_inertial_to_body      = quaternion.from_float_array(state[6:10])
    q_inertial_to_body_init = quaternion.from_float_array(state_init[6:10])
    
    if guid_mode == -1: #Pre-launch
        burn                    = False
        q_inertial_to_target    = q_inertial_to_body_init
    elif guid_mode == 0: #Lift-off and Attitude Hold
        burn                    = True
        q_inertial_to_target    = q_inertial_to_body_init
    elif guid_mode == 1: #Azimuth Align
        target_inclination      = guid_params[0]
        azimuth                 = np.radians(90 + target_inclination)
        q_inertial_to_enu       = get_inertial_to_enu(cart_to_lla(state_init[0:3], r_moon))
        rot_vec                 = quaternion.rotate_vectors(q_inertial_to_enu.conjugate(), np.array([0, 1, 0]))
        q_inertial_to_x_up      = q_inertial_to_enu * quaternion.from_rotation_vector(np.radians(90)*rot_vec).normalized()
        rot_vec                 = quaternion.rotate_vectors(q_inertial_to_x_up.conjugate(), np.array([1, 0, 0]))
        q_inertial_to_target    = q_inertial_to_x_up * quaternion.from_rotation_vector(np.radians(azimuth+180)*rot_vec).normalized()
        burn                    = True
    elif guid_mode == 2: #Orbit Insert
        mass                    = state[13]
        target_inclination      = guid_params[0]
        linear_guid_law         = guid_params[1]
        
        thrust                  = linear_guid_law[0]
        max_time                = linear_guid_law[1]
        accel                   = thrust/mass
        red_rate                = accel/max_time
        vert_accel_cmd          = thrust/mass - red_rate*(t-t_mode2)

        target_inclination      = guid_params[0]
        azimuth                 = np.radians(90 + target_inclination)
        q_inertial_to_enu       = get_inertial_to_enu(cart_to_lla(state_init[0:3], r_moon))
        rot_vec                 = quaternion.rotate_vectors(q_inertial_to_enu.conjugate(), np.array([0, 1, 0]))
        q_inertial_to_x_up      = q_inertial_to_enu * quaternion.from_rotation_vector(np.radians(90)*rot_vec).normalized()
        rot_vec                 = quaternion.rotate_vectors(q_inertial_to_x_up.conjugate(), np.array([1, 0, 0]))
        q_target_to_azi         = q_inertial_to_x_up * quaternion.from_rotation_vector(np.radians(azimuth+180)*rot_vec).normalized()
        rot_vec                 = quaternion.rotate_vectors(q_target_to_azi.conjugate(), np.array([0, 1, 0]))
        pitch_angle             = np.arccos(vert_accel_cmd/accel)
        q_inertial_to_target    = q_target_to_azi * quaternion.from_rotation_vector(pitch_angle*rot_vec).normalized()
        burn                    = True
    elif guid_mode == 3: #Coasting
        burn                    = False
        q_inertial_to_target    = get_inertial_to_lvlh(state)

    return burn, quaternion.as_float_array(q_inertial_to_target)

In [ ]:
def state_dot(t, state, static_params, sc_commands):
    pos                 = state[0:3]
    vel                 = state[3:6]
    q_inertial_to_body  = quaternion.from_float_array(state[6:10])
    omega               = state[10:13]
    mass                = state[13]

    mu      = static_params[0]
    J       = static_params[1]
    m_dry   = static_params[2]
    T_APS   = static_params[3]
    Isp_APS = static_params[4]
    grav    = gravity(pos, mu)

    burn_APS                = sc_commands[0]
    q_inertial_to_target    = sc_commands[1]

    if burn_APS and mass > m_dry:
        T_mag   = T_APS
    else: 
        T_mag   = 0
    
    T_inertial  = quaternion.rotate_vectors(q_inertial_to_body.conjugate(), T_mag*np.array([1, 0, 0]))
    L_body      = get_torque_cmd(state[6:10], omega, q_inertial_to_target)

    p_dot   = vel
    v_dot   = grav + T_inertial/mass
    q_dot   = 1/2 * np.quaternion(0, omega[0], omega[1], omega[2]) * q_inertial_to_body
    alpha   = np.linalg.inv(J) @ (L_body - np.cross(omega,  J @ omega))
    m_dot   = -T_mag / (Isp_APS * 9.81)
    x_dot   = np.hstack((p_dot, v_dot, quaternion.as_float_array(q_dot), alpha, m_dot))
    return x_dot

In [ ]:
def simulate(init_state, times, static_params, mode_switch_states, guidance_params):
    state = init_state
    dt = times[1] - times[0]

    state_hist                  = np.zeros((len(times), 14))
    q_inertial_to_target_hist   = np.zeros((len(times), 4))
    enu_state_hist              = np.zeros((len(times), 6))

    state_hist[0,:]                 = state
    q_inertial_to_target_hist[0]    = state[6:10]
    
    enu_state_hist[0, 0:3]  = convert_inertial_state_to_enu(state[0:3], cart_to_lla(state[0:3], r_moon))
    enu_state_hist[0, 3:6]  = state[3:6]
    idx_events              = []

    t_mode2 = -1

    curr_mode = -1
    for i in range(0, len(times)-1):
        t_now = times[i]
        if np.linalg.norm(state[0:3]) > r_moon:
            guidance_mode = get_guidance_mode(t_now, state, curr_mode, mode_switch_states, t_mode2)
            if guidance_mode != curr_mode:
                idx_events.append(i)
            if t_mode2 == -1 and guidance_mode == 2:
                t_mode2 = t_now

            curr_mode   = guidance_mode
            sc_commands = get_sc_commands(t_now, state, init_state, guidance_mode, guidance_params, t_mode2)
            sol         = solve_ivp(state_dot, t_span=[t_now, t_now+dt],y0=state, 
                                    args=(static_params,sc_commands), method='RK45')

            state = sol.y[:,-1]

        state_hist[i+1]                 = state
        q_inertial_to_target_hist[i+1]  = sc_commands[1]
        enu_state_hist[i+1, 0:3]        = convert_inertial_state_to_enu(state[0:3], cart_to_lla(state_hist[0,0:3], r_moon))
        enu_state_hist[i+1, 3:6]        = state[3:6]
    return state_hist, enu_state_hist, q_inertial_to_target_hist, idx_events

In [ ]:
csm_semimajor   = (101.00e3 + 122.00e3)/2 + r_moon
csm_e           = 0.00568
csm_inc         = 0.00
csm_raan        = 0.00
csm_omega       = 0.00
csm_f           = 0.00

csm_orbital_elements    = np.array([csm_semimajor, csm_e, csm_inc, csm_raan, csm_omega, csm_f])
csm_state_vec           = keplerian_to_state_vec(csm_orbital_elements, mu_moon)


launch_site = np.array([0.67409, 23.47298, 1])
mcmf_pos    = lla_to_cart(launch_site, r_moon)

mcmf_state  = np.hstack((mcmf_pos, np.zeros(3)))

enu_pos     = convert_inertial_state_to_enu(mcmf_state[0:3], launch_site)

q_inertial_to_enu   = get_inertial_to_enu(launch_site)
rot_vec             = quaternion.rotate_vectors(q_inertial_to_enu.conjugate(), np.array([0, 1, 0]))
q_inertial_to_x_up  = q_inertial_to_enu * quaternion.from_rotation_vector(np.radians(90)*rot_vec)
rot_vec             = quaternion.rotate_vectors(q_inertial_to_x_up.conjugate(), np.array([1, 0, 0]))
q_inertial_to_body  = q_inertial_to_x_up * quaternion.from_rotation_vector(np.radians(170)*rot_vec)

omega_body = np.zeros(3)

mass_dry    = 2150.028  # kg
mass        = 4888.18 # kg
J           = np.diag([9230, 4711, 8096])
T_APS       = 15570     # N
Isp_APS     = 311       # s
static_params   = (mu_moon, J, mass_dry, T_APS, Isp_APS)

att_hold_end            = 2      #sec
vertical_rise_end_vel   = 12.192 #m/s
t_orbit_insert          = 450
mode_switch_states      = [att_hold_end, vertical_rise_end_vel, t_orbit_insert]

target_incl         = 0
linear_guid_params  = [T_APS, t_orbit_insert]  
guidance_params     = [csm_inc, linear_guid_params]  # deg

state = np.hstack((mcmf_state, quaternion.as_float_array(q_inertial_to_body), omega_body, mass))
times = np.arange(0, 3000, .1)
state_hist, enu_hist, q_inertial_to_target_hist, idx_events = simulate(state, times, static_params, mode_switch_states, guidance_params)

terminal_state  = state_hist[-1]
kep_final       = state_vec_to_keplerian(terminal_state, mu_moon)
print("CSM Initial Keplerian Elements:")
disp_kep_elem(csm_orbital_elements)
print("LM Final Keplerian Elements:")
disp_kep_elem(kep_final)

In [ ]:
def draw_planet(radius, colorscale):
    # Polar coordinates
    theta   = np.linspace(0, 2.*np.pi, 40)
    phi     = np.linspace(0, np.pi, 40)

    # Convert to cartesian
    x       = radius * np.outer(np.cos(theta), np.sin(phi))
    y       = radius * np.outer(np.sin(theta), np.sin(phi))
    z       = radius * np.outer(np.ones(np.size(theta)), np.cos(phi))

    # Create the planet object
    planet  = go.Surface(x=x, y=y, z=z, opacity=1.0, colorscale=colorscale, showscale=False)
    return planet

def draw_target_axes(pos, orient, scale):
    rot_targ    = quaternion.as_rotation_matrix(quaternion.from_float_array(orient))

    # Plot the target axes
    x_target_x_comps    = [pos[0], pos[0] + rot_targ[0,0]*scale]
    x_target_y_comps    = [pos[1], pos[1] + rot_targ[0,1]*scale]
    x_target_z_comps    = [pos[2], pos[2] + rot_targ[0,2]*scale]
    x_target_obj        = go.Scatter3d(x=x_target_x_comps,
                                       y=x_target_y_comps,
                                       z=x_target_z_comps,
                                       mode="lines", 
                                       line=dict(width=3, color="purple"),
                                       showlegend=False)
    
    y_target_x_comps    = [pos[0], pos[0] + rot_targ[1,0]*scale]
    y_target_y_comps    = [pos[1], pos[1] + rot_targ[1,1]*scale]
    y_target_z_comps    = [pos[2], pos[2] + rot_targ[1,2]*scale]
    y_target_obj        = go.Scatter3d(x=y_target_x_comps,
                                       y=y_target_y_comps,
                                       z=y_target_z_comps,
                                       mode="lines", 
                                       line=dict(width=3, color="purple"),
                                       showlegend=False)
    
    z_target_x_comps    = [pos[0], pos[0] + rot_targ[2,0]*scale]
    z_target_y_comps    = [pos[1], pos[1] + rot_targ[2,1]*scale]
    z_target_z_comps    = [pos[2], pos[2] + rot_targ[2,2]*scale]
    z_target_obj        = go.Scatter3d(x=z_target_x_comps,
                                       y=z_target_y_comps,
                                       z=z_target_z_comps,
                                       mode="lines", 
                                       line=dict(width=3, color="purple"),
                                       showlegend=False)
    
    # Plot cones at the ends of the target axes
    x_target_cone_x     = [pos[0] + rot_targ[0,0]*scale]
    x_target_cone_y     = [pos[1] + rot_targ[0,1]*scale]
    x_target_cone_z     = [pos[2] + rot_targ[0,2]*scale]
    x_target_cone_u     = [rot_targ[0,0]*scale*.3]
    x_target_cone_v     = [rot_targ[0,1]*scale*.3]
    x_target_cone_w     = [rot_targ[0,2]*scale*.3]
    x_targ_cone         = go.Cone(x=x_target_cone_x,
                                  y=x_target_cone_y,
                                  z=x_target_cone_z,
                                  u=x_target_cone_u,
                                  v=x_target_cone_v,
                                  w=x_target_cone_w,
                                  anchor = "tail",
                                  hoverinfo =  None,
                                  colorscale = [[0, "red"], [1, "red"]],
                                  showscale = False, showlegend=False)
    
    y_target_cone_x     = [pos[0] + rot_targ[1,0]*scale]
    y_target_cone_y     = [pos[1] + rot_targ[1,1]*scale]
    y_target_cone_z     = [pos[2] + rot_targ[1,2]*scale]
    y_target_cone_u     = [rot_targ[1,0]*scale*.3]
    y_target_cone_v     = [rot_targ[1,1]*scale*.3]
    y_target_cone_w     = [rot_targ[1,2]*scale*.3]
    y_targ_cone         = go.Cone(x=y_target_cone_x,
                                  y=y_target_cone_y,
                                  z=y_target_cone_z,
                                  u=y_target_cone_u,
                                  v=y_target_cone_v,
                                  w=y_target_cone_w,
                                  anchor = "tail",
                                  hoverinfo =  None,
                                  colorscale = [[0, "green"], [1, "green"]],
                                  showscale = False, showlegend=False)
                          
    z_targ_cone_x       = [pos[0] + rot_targ[2,0]*scale]
    z_targ_cone_y       = [pos[1] + rot_targ[2,1]*scale]
    z_targ_cone_z       = [pos[2] + rot_targ[2,2]*scale]
    z_targ_cone_u       = [rot_targ[2,0]*scale*.3]
    z_targ_cone_v       = [rot_targ[2,1]*scale*.3]
    z_targ_cone_w       = [rot_targ[2,2]*scale*.3]
    z_targ_cone         = go.Cone(x=z_targ_cone_x,
                                  y=z_targ_cone_y,
                                  z=z_targ_cone_z,
                                  u=z_targ_cone_u,
                                  v=z_targ_cone_v,
                                  w=z_targ_cone_w,
                                  anchor = "tail",
                                  hoverinfo =  None,
                                  colorscale = [[0, "blue"], [1, "blue"]],
                                  showscale = False, showlegend=False)
    
    return [x_target_obj, y_target_obj, z_target_obj, x_targ_cone, y_targ_cone, z_targ_cone]

def draw_spacecraft_axes(pos, orient, scale):
    # Convert the quaternion to a rotation matrix to get the axes
    rot         = quaternion.as_rotation_matrix(quaternion.from_float_array(orient))

    # Create the data for the plot
    # Plot the spacecraft axes
    x_x_comps   = [pos[0], pos[0] + rot[0,0]*scale]
    x_y_comps   = [pos[1], pos[1] + rot[0,1]*scale]
    x_z_comps   = [pos[2], pos[2] + rot[0,2]*scale]
    x_axis_obj  = go.Scatter3d(x=x_x_comps,
                               y=x_y_comps,
                               z=x_z_comps,
                               mode="lines", 
                               line=dict(width=3, color="red"), showlegend=False)
    
    y_x_comps   = [pos[0], pos[0] + rot[1,0]*scale]
    y_y_comps   = [pos[1], pos[1] + rot[1,1]*scale]
    y_z_comps   = [pos[2], pos[2] + rot[1,2]*scale]
    y_axis_obj  = go.Scatter3d(x=y_x_comps,
                               y=y_y_comps,
                               z=y_z_comps,
                               mode="lines", 
                               line=dict(width=3, color="green"), showlegend=False)
    
    z_x_comps   = [pos[0], pos[0] + rot[2,0]*scale]
    z_y_comps   = [pos[1], pos[1] + rot[2,1]*scale]
    z_z_comps   = [pos[2], pos[2] + rot[2,2]*scale]
    z_axis_obj  = go.Scatter3d(x=z_x_comps,
                               y=z_y_comps,
                               z=z_z_comps,
                               mode="lines", 
                               line=dict(width=3, color="blue"), showlegend=False)
    return [x_axis_obj, y_axis_obj, z_axis_obj]

def draw_traj(posits, idx, colorscale):
    spacecraft_traj  = go.Scatter3d(x=posits[0:idx,0],
                                    y=posits[0:idx,1],
                                    z=posits[0:idx,2],
                                    mode="lines",
                                    line=dict(width=5, colorscale=colorscale, 
                                              color=np.arange(0, idx, 1), 
                                              showscale=False), showlegend=False)
    return [spacecraft_traj]

def states_for_plot(sat_pos, sat_orient, sat_quat_targ):
    sat_state_plot = np.zeros((len(sat_pos), 11))
    sat_state_plot[:,0:3]   = sat_pos
    sat_state_plot[:,3:7]   = sat_orient
    sat_state_plot[:,7:11]  = sat_quat_targ

    return sat_state_plot

def get_3d_frame_data(plot_states, idx, scale):
    # Unpack input data
    pos         = plot_states[idx, 0:3]
    orient      = plot_states[idx, 3:7]
    quat_targ   = plot_states[idx, 7:11]

    # Create the spacecraft trajectory
    spacecraft_traj = draw_traj(plot_states[:, 0:3], idx, "Oranges")

    # Plot the spacecraft axes
    sc_axes    = draw_spacecraft_axes(pos, orient, scale)
    
    # Plot the target axes
    targ_axes   = draw_target_axes(pos, quat_targ, scale)

    # Add all the objects to the list
    objs = spacecraft_traj
    objs.extend(targ_axes)
    objs.extend(sc_axes)
    return objs

def create_3d_animation(fig, plotformat, sat_state_plot, update_rate):
    # Unpack the plot format
    title       = plotformat[0]
    scale       = plotformat[1]
    xaxis       = plotformat[2]
    yaxis       = plotformat[3]
    zaxis       = plotformat[4]

    # Get the data for each frame
    fig.update(frames=[go.Frame(data=get_3d_frame_data(sat_state_plot, i*update_rate, scale), 
                                traces=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]) 
                                for i in range(1, round(len(sat_state_plot)/update_rate))])
    
    fig.update_layout(title=dict(text=title),
                      font=dict(family="Courier New, monospace", color="RebeccaPurple"),
                      transition = {'duration': 1},
                      scene=dict(aspectmode="cube",
                      xaxis=xaxis, 
                      yaxis=yaxis, 
                      zaxis=zaxis),
                                 updatemenus=[dict(type="buttons",buttons=[dict(label="Play",method="animate",
                                                args=[None, {"frame": {"duration": 1, "redraw": True}}])])],
                        showlegend=False)
    return fig

def create_inertial_animimation(sat_pos, sat_orient, sat_targ_orient, update_rate):
    max_range = np.max(np.abs(sat_pos))

    min_axis = -1.2*max_range
    max_axis = 1.2*max_range
    scale   = max_axis / 7
    xaxis   = dict(range=[min_axis, max_axis], title=dict(text="X [m]"))
    yaxis   = dict(range=[min_axis, max_axis], title=dict(text="Y [m]"))
    zaxis   = dict(range=[min_axis, max_axis], title=dict(text="Z [m]"))

    plotformat = [f"ECI Trajectory", scale, xaxis, yaxis, zaxis]

    sat_state_plot  = states_for_plot(sat_pos, sat_orient, sat_targ_orient)

    frame_data  = get_3d_frame_data(sat_state_plot, 0, scale)
    frame_data.append(draw_planet(r_moon, "Greys"))
    fig         = go.Figure(data=frame_data)

    animation = create_3d_animation(fig, plotformat, sat_state_plot, update_rate)
    return animation

def create_enu_fig(times, sat_state, idx_events):
    max_range = np.max(np.abs(sat_state[:,0:3]))

    min_axis = -1.2*max_range
    max_axis = 1.2*max_range

    xaxis   = dict(range=[min_axis, max_axis], title=dict(text="East [m]"))
    yaxis   = dict(range=[min_axis, max_axis], title=dict(text="North [m]"))
    zaxis   = dict(range=[0, 2*max_axis], title=dict(text="Up [m]"))

    fig = go.Figure(data=[go.Scatter3d(x=sat_state[:,0], 
                                       y=sat_state[:,1], 
                                       z=sat_state[:,2], 
                                       mode="lines", 
                                       name="Trajectory",
                                       line=dict(width=8, colorscale="Oranges", 
                                                 color=np.linalg.norm(sat_state[:,3:6], axis=1), 
                                                 showscale=False))])
    
    fig.add_trace(go.Scatter3d(x=[sat_state[idx_events[0], 0]],
                               y=[sat_state[idx_events[0], 1]],
                               z=[sat_state[idx_events[0], 2]],
                               mode="markers",
                               marker=dict(size=5),
                               name=f"Lift-Off<br>T+{times[idx_events[0]]:.2f}sec<br>Altitude: {np.linalg.norm(sat_state[idx_events[0], 0:3]):.2f}m<br>Velocity: {np.linalg.norm(sat_state[idx_events[0], 3:6]):.2f}m/s"))
    
    fig.add_trace(go.Scatter3d(x=[sat_state[idx_events[1], 0]],
                               y=[sat_state[idx_events[1], 1]],
                               z=[sat_state[idx_events[1], 2]],
                               mode="markers",
                               marker=dict(size=5),
                               name=f"Start Azimuth Align<br>T+{times[idx_events[1]]:.2f}sec<br>Altitude: {np.linalg.norm(sat_state[idx_events[1], 0:3]):.2f}m<br>Velocity: {np.linalg.norm(sat_state[idx_events[1], 3:6]):.2f}m/s"))
    
    fig.add_trace(go.Scatter3d(x=[sat_state[idx_events[2], 0]],
                               y=[sat_state[idx_events[2], 1]],
                               z=[sat_state[idx_events[2], 2]],
                               mode="markers",
                               marker=dict(size=5),
                               name=f"Start Orbit Insertion<br>T+{times[idx_events[2]]:.2f}sec<br>Altitude: {np.linalg.norm(sat_state[idx_events[2], 0:3]):.2f}m<br>Velocity: {np.linalg.norm(sat_state[idx_events[2], 3:6]):.2f}m/s"))
    
    fig.add_trace(go.Scatter3d(x=[sat_state[idx_events[3], 0]],
                               y=[sat_state[idx_events[3], 1]],
                               z=[sat_state[idx_events[3], 2]],
                               mode="markers",
                               marker=dict(size=5),
                               name=f"Coasting<br>T+{times[idx_events[3]]:.2f}sec<br>Altitude: {np.linalg.norm(sat_state[idx_events[3], 0:3]):.2f}m<br>Velocity: {np.linalg.norm(sat_state[idx_events[3], 3:6]):.2f}m/s"))
    
    fig.update_layout(title = "ENU Trajectory", 
                      scene=dict(aspectmode="cube",xaxis=xaxis, yaxis=yaxis, zaxis=zaxis))
    
    return fig

In [ ]:
animation = create_inertial_animimation(state_hist[:,0:3], state_hist[:,6:10], q_inertial_to_target_hist, len(times)//100)
animation.show()

In [ ]:
enu_fig = create_enu_fig(times[:idx_events[-1]+1], enu_hist[:idx_events[-1]+1, :], idx_events)
enu_fig.show()

In [ ]:
states_powered = state_hist[:idx_events[-1]+1,:]
times_powered = times[:idx_events[-1]+1]

fig = go.Figure(data=[go.Scatter(x=times_powered, y=np.linalg.norm(states_powered[:,0:3], axis=1), mode="lines", name="Altitude [m]")])
fig.add_vline(x=times_powered[idx_events[0]], line_dash="solid", line_color="red", annotation_text="<b>Lift-Off</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[2]], line_dash="dot", line_color="purple", annotation_text="<b>Start Orbit Insertion</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[3]], line_dash="dash", line_color="orange", annotation_text="<b>Start Coasting</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft Altitude</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Altitude [m]</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_powered, y=np.linalg.norm(states_powered[:,3:6], axis=1), mode="lines", name="Velocity [m/s]")])
fig.add_vline(x=times_powered[idx_events[0]], line_dash="solid", line_color="red", annotation_text="<b>Lift-Off</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[2]], line_dash="dot", line_color="purple", annotation_text="<b>Start Orbit Insertion</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[3]], line_dash="dash", line_color="orange", annotation_text="<b>Start Coasting</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft Velocity</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Velocity [m/s]</b>", title_font=dict(size=14))
fig.show()

error_quat = get_error_quats(states_powered[:,6:10], q_inertial_to_target_hist[:idx_events[-1]+1])
fig = go.Figure(data=[go.Scatter(x=times_powered, y=error_quat[:,1], mode="lines", name=r"$\delta q_x$")])
fig.add_trace(go.Scatter(x=times_powered, y=error_quat[:,2], mode="lines", name=r"$\delta q_y$"))
fig.add_trace(go.Scatter(x=times_powered, y=error_quat[:,3], mode="lines", name=r"$\delta q_z$"))
fig.add_vline(x=times_powered[idx_events[1]], line_dash="dash", line_color="green", annotation_text="<b>Start Azimuth Align</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[2]], line_dash="dot", line_color="purple", annotation_text="<b>Start Orbit Insertion</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[3]], line_dash="dash", line_color="orange", annotation_text="<b>Start Coasting</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Error Quaternion</b>", font=dict(size=16))
fig.update_xaxes(title= "<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Error Quaternion Value</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_powered, y=states_powered[:,10], mode="lines", name=r"$\omega_x$")])
fig.add_trace(go.Scatter(x=times_powered, y=states_powered[:,11], mode="lines", name=r"$\omega_y$"))
fig.add_trace(go.Scatter(x=times_powered, y=states_powered[:,12], mode="lines", name=r"$\omega_z$"))
fig.add_vline(x=times_powered[idx_events[1]], line_dash="dash", line_color="green", annotation_text="<b>Start Azimuth Align</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[2]], line_dash="dot", line_color="purple", annotation_text="<b>Start Orbit Insertion</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[3]], line_dash="dash", line_color="orange", annotation_text="<b>Start Coasting</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft Angular Velocity</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Angular Velocity [rad/s]</b>", title_font=dict(size=14))
fig.show()

fig = go.Figure(data=[go.Scatter(x=times_powered, y=states_powered[:,13], mode="lines", name="Mass [kg]")])
fig.add_vline(x=times_powered[idx_events[0]], line_dash="solid", line_color="red", annotation_text="<b>Lift-Off</b>", annotation_position="top right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[2]], line_dash="dot", line_color="purple", annotation_text="<b>Start Orbit Insertion</b>", annotation_position="bottom right", annotation_font=dict(size=14))
fig.add_vline(x=times_powered[idx_events[3]], line_dash="dash", line_color="orange", annotation_text="<b>Start Coasting</b>", annotation_position="top left", annotation_font=dict(size=14))
fig.update_layout(title="<b>Spacecraft Mass</b>", font=dict(size=16))
fig.update_xaxes(title="<b>Time [s]</b>", title_font=dict(size=14))
fig.update_yaxes(title="<b>Mass [kg]</b>", title_font=dict(size=14))
fig.show()